In [1]:
import os, re, h5py
import numpy as np
import tifffile as Tiff
from PIL import Image
from skimage import io
from skimage import exposure
from skimage.viewer import ImageViewer
from subprocess import call
from natsort import natsorted
import subprocess
import psycopg2
import warnings; warnings.simplefilter('ignore')

/Users/OGN/anaconda/envs/py36/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


### Import HDF5 contents from the Randlett, Burgess, and Heisemeyer lines (Anatomy, ZBB, MH)


In [2]:
# from IPython.display import Image

HDF5s = []
HDF5s.append(h5py.File('AnatomyLabelDatabase.hdf5', 'r'))
HDF5s.append(h5py.File('ZBBDatabase.hdf5', 'r'))
HDF5s.append(h5py.File('ZBBDatabase2.hdf5', 'r'))
HDF5s.append(h5py.File('Harvard_MH_Lines.hdf5', 'r'))

OSError: Unable to open file (Unable to lock file, errno = 35, error message = 'resource temporarily unavailable')

#### Print the names of all lines (stacks)


In [3]:
print('Anatomy:\n', [k for k in HDF5s[0].keys()], '\n')
print('ZBB:\n', [k for k in HDF5s[1].keys()], '\n')
print('ZBB2:\n', [k for k in HDF5s[2].keys()], '\n')
print('MH:\n', [k for k in HDF5s[3].keys()], '\n')

Anatomy:
 ['6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish', 'Anti-5HT_6dpf_MeanImageOf40Fish', 'Anti-Gad67_6dpf_MeanImageOf17Fish', 'Anti-GlyR_6dpf_MeanImageOf160Fish', 'Anti-TH_6dpf_MeanImageOf10Fish', 'Anti-Zn12(Hnk-1)_6dpf_MeanImageOf9Fish', 'Anti-Zn1_6dpf_MeanImageOf10Fish', 'Anti-Znp1(Synaptotagmin2)_6dpf_MeanImageOf118Fish', 'Anti-Zrf1(GFAP)_6dpf_MeanImageOf5Fish', 'Anti-Zrf2_6dpf_MeanImageOf5Fish', 'Anti-tERK_6dpf_MeanImageOf193Fish', 'Elavl3-GCaMP5G_6dpf_MeanImageOf7Fish', 'Elavl3-H2BRFP_6dpf_MeanImageOf10Fish', 'EtVmat2-GFP_6dpf_MeanImageOf55Fish', 'Gad1b-GFP_6dpf_MeanImageOf10Fish', 'Glyt2-GFP_6dpf_MeanImageOf13Fish', 'Hcrt-RFP_6dpf_MeanImageOf15Fish', 'Isl1-GFP_6dpf_MeanImageOf17Fish', 'Isl2bGal4-uasDendra_6dpf_MeanImageOf8Fish', 'Olig2-GFP_6dpf_MeanImageOf12Fish', 'Otpb.A-Gal4-UAS-GCaMP_6-7dpf_MeanImageOf19Fish', 'Otpb.A-Nsfb-GFP_6dpf_MeanImageOf10Fish', 'Oxtl-GFP_6dpf_MeanImageOf68Fish', 'Pet1-GFP_6dpf_MeanImageOf13Fish', 'Ptf1aGal4-uasKaede_6dpf_MeanImageOf11Fish', 'Qrf

#### Copy single stack from one HDF5

In [12]:
data = HDF5s[0].get('Otpb.A-Nsfb-GFP_6dpf_MeanImageOf10Fish')

IndexError: list index out of range

#### Copy single slice from one stack

In [5]:
image = data[75]
# tiff.imsave('Otpb_a-Nsfb75.tiff', image)
# io.imsave('Otpb_a-Nsfb75.jpeg', image)

#### Check brightness stats (all images grayscale)

In [6]:
print(np.mean(data))
np.max(data)

4645.82173008


65535

In [7]:
data = HDF5s[2].get('s356t_brn3c-GFP')
print(np.mean(data))

56.0304036003


### Test basic image manipulation with Subprocess >> Imagemagick c.l. tools

In [8]:
adj_data0 = data[90]
try:
    io.imsave('Test/test_16bit_jpeg0.jpg', adj_data0)
except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'Test/test_16bit_jpeg0.jpg'


#### Image created ^ Now modify with 'convert'

In [38]:
try:
    subprocess.check_output(["convert", "Test/test_16bit_jpeg0.jpg", "-format", "jpg", "-level", "0%, 50%", "Test/test_16bit_jpeg1.jpg"], stderr=subprocess.STDOUT)
except Exception as e:
    print(e)

Command '['convert', 'Test/test_16bit_jpeg0.jpg', '-format', 'jpg', '-level', '0%, 50%', 'Test/test_16bit_jpeg1.jpg']' returned non-zero exit status 1.


#### Display image

In [39]:
try:
    Image("Test/test_16bit_jpeg0.jpg")
except Exception as e:
    print(e)

'module' object is not callable


### Export images from HDF5 files

#### Export tiff stacks from HDF5 files

In [40]:
for n, h in enumerate(HDF5s):
    for k, key in enumerate(h.keys()):
        stack = h.get(key)
        stack_name = str(re.split('_6(.*)dpf', key)[0]) # applies only to Anatomy and MH lines, chops dpf suffix
        
        try:
            int(stack_name)                             # applies only to MH Lines
            stack_name = 'MH_{}-Gal4'.format(stack_name)
        except:
            pass
        
        if n == 2: # assuming ZBBDatabase2 is HDF5s[2]
            stack_name = 'ZBB_' + stack_name
        else: 
            continue
        
        tiff_path = '../app/assets/images/4-TiffStacks/{}.tif'.format(stack_name) # assuming cwd is db/
        if not os.path.isfile(tiff_path):
            print(tiff_path)
            Tiff.imsave(tiff_path, stack)
        

In [10]:
AN = HDF5s[0]
ZBB1 = HDF5s[1]
ZBB2 = HDF5s[2]
MH = HDF5s[3]

#### Export line JPGs from Anatomy HFD5

In [13]:
for k, key in enumerate(AN.keys()):
    stack = AN.get(key)
    stack_name = key.split('_')[0]
    folder_path = '../app/assets/images/0-Lines/%s/' % (stack_name)
    print(folder_path)
    print(np.mean(stack))

    try:
        os.makedirs(folder_path)
    except OSError as exception:
        print(exception)
        
    for i, img in enumerate(stack):
        img_path = '../app/assets/images/0-Lines/%s/%s-%i.jpg' % (stack_name, key, i)
        io.imsave(img_path, stack[i])
        subprocess.check_output(["convert", img_path, "-format", "jpg", "-level", "0%, 50%", img_path], stderr=subprocess.STDOUT)


NameError: name 'AN' is not defined

#### Export line JPGs from first ZBB HDF5

In [ ]:
for k, key in enumerate(ZBB1.keys()):
    stack = ZBB1.get(key)
    stack_name = key
    folder_path = '../app/assets/images/0-Lines/%s/' % (stack_name)
    print(folder_path)
    mean_intensity = np.mean(stack)
    
    if mean_intensity >= 1000:
        white_point = 50
    elif mean_intensity >= 100 and mean_intensity < 1000:
        white_point = 40
    else:
        white_point = 30

    try:
        os.makedirs(folder_path)
    except OSError as exception:
        print(exception)
    
    print(white_point)
    for i, img in enumerate(stack):
        img_path = '../app/assets/images/0-Lines/%s/%s-%i.jpg' % (stack_name, key, i)
        #io.imsave(img_path, stack[i])
        #subprocess.check_output(["convert", img_path, "-format", "jpg", "-level", "0%%, %i%%" % (white_point) , img_path], stderr=subprocess.STDOUT)    
        

#### Export line JPGs from second ZBB HDF5

In [ ]:
for k, key in enumerate(ZBB2.keys()):
    stack = ZBB2.get(key)
    stack_name = 'ZBB_' + key
    folder_path = '../app/assets/images/0-Lines/%s/' % (stack_name)
    mean_intensity = np.mean(stack)
    print (stack_name, mean_intensity)

    if mean_intensity >= 1000: pho
        white_point = 50
    elif mean_intensity >= 100 and mean_intensity < 1000:
        white_point = 40
    else:
        white_point = 30

    try:
        os.makedirs(folder_path)
    except OSError as exception:
        print(exception)
    
    for i, img in enumerate(stack):
        img_path = '../app/assets/images/0-Lines/%s/%s-%i.jpg' % (stack_name, stack_name, i)
        io.imsave(img_path, stack[i])
        subprocess.check_output(["convert", img_path, "-format", "jpg", "-level", "0%%, %i%%" % (white_point) , img_path], stderr=subprocess.STDOUT)    
        

#### Export line JPGs from Harvard_MH HFD5 without creating database entries

In [6]:
for k, key in enumerate(MH.keys()):
    stack = MH.get(key)
    stack_number = key.split('_')[0]
    stack_name = 'MH_{}-Gal4'.format(stack_number)
    folder_path = '../app/assets/images/0-Lines/%s/' % (stack_name)
    mean_intensity = np.mean(stack)
    print (stack_name, mean_intensity)

    try:
        os.makedirs(folder_path)
    except OSError as exception:
        pass
    
    white_point = 40

    for i, img in enumerate(stack):
        img_path = '{}/{}-{}.jpg'.format(folder_path, stack_name, i)
        img = np.flipud(img)
        io.imsave(img_path, img)
            # skip to keep native whitelevels
        #subprocess.check_output(["convert", img_path, "-format", "jpg", "-level", "0%%, %i%%" % (white_point) , img_path], stderr=subprocess.STDOUT)

MH_13319-Gal4 2812.68795899
MH_15576-Gal4 3574.26017525
MH_16659-Gal4 11615.5238966
MH_16715-Gal4 9433.42074963
MH_16913-Gal4 10686.3600645
MH_16914-Gal4 1450.09333567
MH_16915-Gal4 6869.44971984
MH_16916-Gal4 2636.40262084
MH_16946-Gal4 5873.45485924
MH_17056-Gal4 3005.27221835
MH_17767-Gal4 2076.90699749
MH_17768-Gal4 2019.05356542
MH_17776-Gal4 6648.48493657
MH_17777-Gal4 3427.17666903
MH_17787-Gal4 4313.95343514
MH_17845-Gal4 2636.49697203
MH_17846-Gal4 4107.92464891
MH_18107-Gal4 3221.95892779
MH_18108-Gal4 2774.80982257
MH_18327-Gal4 5405.86273808
MH_18329-Gal4 3135.75183934
MH_19322-Gal4 5112.56925594
MH_19566-Gal4 3466.57842414
MH_20360-Gal4 2407.11234121
MH_20361-Gal4 2513.67460918
MH_20454-Gal4 1228.75951744
MH_20552-Gal4 4935.75330039
MH_20553-Gal4 1847.23119936
MH_20638-Gal4 2387.89796152
MH_20639-Gal4 1210.14545137
MH_20640-Gal4 1347.12983667
MH_20641-Gal4 1341.35001877
MH_20642-Gal4 1790.26426982
MH_20643-Gal4 2817.13097499
MH_20644-Gal4 3250.35008935
MH_20646-Gal4 2192.1

#### Export line JPGs from Generic tif

In [2]:
os.getcwd()

'/Users/OGN/1Projects/ZBrain2/db'

In [5]:
tiffFileName = "HGn39D"
tiff = Tiff.imread('..app/assets/images/4-TiffStacks/%s.tif'% tiffFileName)
print(tiff.shape)

(138, 621, 1406)


In [8]:
#tiff = np.squeeze(tiff, axis=0)
#tiff = np.squeeze(tiff, axis=1)
#print(tiff.shape)
os.mkdir('../app/assets/images/0-Lines/{0}'.format(tiffFileName))

for i, image in enumerate(tiff):
    img_path = '../app/assets/images/0-Lines/{0}/{1}-{2}.jpg'.format(tiffFileName, tiffFileName, i)
    io.imsave(img_path, image)

### Create database entries for exported images

#### Connect to Postgresql db

In [9]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

#### Initialize '*images*' table

In [10]:
create_table_command = (
"""
    DROP TABLE images;
    CREATE TABLE images (
        image_id SERIAL PRIMARY KEY,
        line_name TEXT NOT NULL,
        line_id INT NOT NULL,
        image_path TEXT NOT NULL
    );
""")

In [11]:
try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

#### Iterate stack directories and create database entries for each JPG

In [12]:
stacks = sorted([d for d in os.listdir('../app/assets/images/0-Lines') if not d.startswith('.')])
stacks

['0-Reference-Brain',
 '6.7FRhcrtR-Gal4-uasKaede',
 'Anti-5HT',
 'Anti-Gad67',
 'Anti-GlyR',
 'Anti-TH',
 'Anti-Zn1',
 'Anti-Zn12(Hnk-1)',
 'Anti-Znp1(Synaptotagmin2)',
 'Anti-Zrf1(GFAP)',
 'Anti-Zrf2',
 'Anti-tERK',
 'Elavl3-GCaMP5G',
 'Elavl3-H2BRFP',
 'EtVmat2-GFP',
 'Gad1b-GFP',
 'Glyt2-GFP',
 'HGn39D',
 'Hcrt-RFP',
 'Isl1-GFP',
 'Isl2bGal4-uasDendra',
 'MH_13319-Gal4',
 'MH_15576-Gal4',
 'MH_16659-Gal4',
 'MH_16715-Gal4',
 'MH_16913-Gal4',
 'MH_16914-Gal4',
 'MH_16915-Gal4',
 'MH_16916-Gal4',
 'MH_16946-Gal4',
 'MH_17056-Gal4',
 'MH_17767-Gal4',
 'MH_17768-Gal4',
 'MH_17776-Gal4',
 'MH_17777-Gal4',
 'MH_17787-Gal4',
 'MH_17845-Gal4',
 'MH_17846-Gal4',
 'MH_18107-Gal4',
 'MH_18108-Gal4',
 'MH_18327-Gal4',
 'MH_18329-Gal4',
 'MH_19322-Gal4',
 'MH_19566-Gal4',
 'MH_20360-Gal4',
 'MH_20361-Gal4',
 'MH_20454-Gal4',
 'MH_20552-Gal4',
 'MH_20553-Gal4',
 'MH_20638-Gal4',
 'MH_20639-Gal4',
 'MH_20640-Gal4',
 'MH_20641-Gal4',
 'MH_20642-Gal4',
 'MH_20643-Gal4',
 'MH_20644-Gal4',
 'MH_20646-

In [13]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [14]:
counter = 0
for i, stack_name in enumerate(stacks, 100):
    folder_path = '../app/assets/images/0-Lines/%s/' % stack_name
    rel_folder_path = '%s/' % stack_name
    stack = natsorted([i for i in os.listdir(folder_path) if not i.startswith('.')])
    stack_id = i
    
    for i, s in enumerate(stack):
        rel_img_path = rel_folder_path + s
        insert_command = ("INSERT INTO images (image_id, line_name, line_id, image_path) values (%i, '%s', %i, '%s')" 
                % (counter, stack_name, stack_id, rel_img_path))
        
        try:
            insert_command
            cursor.execute(insert_command)
            counter += 1
        except Exception as e:
            print(e)

#### Check that appropriate number rows, and appropriate row content

In [15]:
cursor.execute('SELECT * FROM images')
rows = cursor.rowcount
print(rows)

42642


In [16]:
cursor.close()
conn.commit()

In [17]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [18]:
cursor.execute('SELECT * FROM images order by image_id')

In [19]:
rows = cursor.fetchall()
for row in rows[11100:11200]:
    print(row)

(11100, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-60.jpg')
(11101, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-61.jpg')
(11102, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-62.jpg')
(11103, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-63.jpg')
(11104, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-64.jpg')
(11105, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-65.jpg')
(11106, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-66.jpg')
(11107, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-67.jpg')
(11108, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-68.jpg')
(11109, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-69.jpg')
(11110, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-70.jpg')
(11111, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-71.jpg')
(11112, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-72.jpg')
(11113, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-73.jpg')
(11114, 'ZBB_ath5-GFP', 180, 'ZBB_ath5-GFP/ZBB_ath5-GFP-74.jpg')
(11115, 'ZBB_ath5-GFP', 1